# Import dependencies

In [ ]:
# Base libraries
import os
import pandas as pd
import numpy as np
import regex as re
from dotenv import load_dotenv

# youtube libraries
from youtube_transcript_api import YouTubeTranscriptApi

# summary libraries
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer  # Cambiado a LsaSummarizer

# Load Data

In [ ]:
file = "your_file.csv"
df = pd.read_csv(file)
df.info()

# Get video trasncript

In [ ]:
def transcript_video(video_id):
    # Fetch the transcript for the given video ID
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['es'])
    return transcript

def transcript_to_text(transcript):
    # Concatenate all snippets into a single string
    t = ''
    for snippet in transcript:
        t += snippet['text'] + '. '
    t = t.replace('  ',' ')
    return t

In [ ]:
def resumen_sumy(texto, num_oraciones=5):
    # Verificar si el texto está vacío o muy corto
    if not texto.strip():
        return "El texto de entrada está vacío."
    
    parser = PlaintextParser.from_string(texto, Tokenizer("spanish"))
    summarizer = LsaSummarizer()  # Cambiar el algoritmo si es necesario
    resumen = summarizer(parser.document, num_oraciones)
    return " ".join(str(oracion) for oracion in resumen)

In [ ]:
def get_video_transcript(): 
    for index, row in df[df['summary_Sumy'].isna()].iterrows():
        if index % 10 == 0:
            print(f"{index} videos processed...")
        try: 
            transcript = transcript_video(row['video_id'])
            text = transcript_to_text(transcript)
            df.at[index, 'summary_Sumy'] = resumen_sumy(text)
        except Exception as e:
            print(f"Error processing video ID {row['video_id']}: {e}")
            df.at[index, 'summary_Sumy'] = 'resumen no disponible'

In [ ]:
get_video_transcript()

In [ ]:
df.to_csv(file, index=False)